# Libraries

In [3]:
from pyomo.environ import *

# Food Manufacture 1

In [11]:
mdl = AbstractModel()

## Sets

$O$ Oils

$T$ Time periods

In [12]:
mdl.O = Set()
mdl.T = Set()

## Parameters

$h_o$ Hardness of oil o

$g_{t,o}$ Purchase price of oil o in period t

$p$ Unit sales price of final product per ton

$q$ Storage cost per product and period

In [13]:
mdl.h = Param(mdl.O)
mdl.g = Param(mdl.T, mdl.O)
mdl.p = Param(within=NonNegativeReals)
mdl.q = Param(within=NonNegativeReals)

## Variables

$b_{t,o}$ Oil o bought in period t

$u_{t,o}$ Oil o used in period t

$s_{t,o}$ Oil o stored in period t

$f_t$ Food produced in period t

$b_{t,o},u_{t,o},s_{t,o}\geq 0\:\forall t\in T,\forall o\in O$

$f_t\geq 0\:\forall t\in T$

In [14]:
mdl.b = Var(mdl.T, mdl.O, domain=NonNegativeReals)
mdl.u = Var(mdl.T, mdl.O, domain=NonNegativeReals)
mdl.s = Var(mdl.T, mdl.O, bounds=(0, 1000), domain=NonNegativeReals)
mdl.f = Var(mdl.T, domain=NonNegativeReals)

## Objective Function

$\max\sum\limits_{t\in T}(p*f_t)-\sum\limits_{t\in T}\sum\limits_{o\in O}(g_{t,o}*b_{t,o}+q*s_{t,o})$

In [15]:
def max_profit(mdl):
    Revenue = sum(mdl.p * mdl.f[t] for t in mdl.T)
    RawCost = sum(mdl.g[t, o] * mdl.b[t, o] for t in mdl.T for o in mdl.O)
    StorageCost = sum(mdl.q * mdl.s[t, o] for t in mdl.T for o in mdl.O)
    return Revenue - RawCost - StorageCost

mdl.OBJ = Objective(rule=max_profit, sense=maximize)

## Constraints

**balance**

$\sum\limits_{o\in O}u_{t,o}=f_t\:\forall t\in T$

$500+b_{t_0,o}=u_{t_0}+s_{t_0,o}\:\forall o\in O$

$s_{t-1,o}+b_{t,o}=u_{t,o}+s_{t,o}\:\forall t\in T\setminus{t_0}, \forall o\in O$

$s_{t_e,o}=500\:\forall o\in O$

**production**

$\sum\limits_{o\in V}u_{t,o}\leq 200\:\forall t\in T$

$\sum\limits_{o\in N}u_{t,o}\leq 250\:\forall t\in T$

**quality**

$3*f_t\leq\sum\limits_{o\in O}h_o*u_{t,o}\leq 6*f_t$

In [16]:
def material_balance_expr(mdl, t):
    return sum(mdl.u[t, o] for o in mdl.O) == mdl.f[t]

mdl.material_balance_con = Constraint(mdl.T, rule=material_balance_expr)

def initial_balance_expr(mdl, o):
    return 500 + mdl.b[1, o] == mdl.u[1, o] + mdl.s[1, o]

mdl.initial_balance_con = Constraint(mdl.O, rule=initial_balance_expr)

def flow_balance_expr(mdl, t, o):
    if t >= 2:
        return mdl.s[t-1, o] + mdl.b[t, o] == mdl.u[t, o] + mdl.s[t, o]
    return Constraint.Skip

mdl.flow_balance_con = Constraint(mdl.T, mdl.O, rule=flow_balance_expr)

def end_balance_expr(mdl, o):
    return mdl.s[6, o] == 500

mdl.end_balance_con = Constraint(mdl.O, rule=end_balance_expr)

def veg_ub_expr(mdl, t):
    return sum(mdl.u[t, o] for o in mdl.O if 'VEG' in o) <= 200

mdl.veg_ub_con = Constraint(mdl.T, rule=veg_ub_expr)

def nonveg_ub_expr(mdl, t):
    return sum(mdl.u[t, o] for o in mdl.O if 'OIL' in o) <= 250

mdl.nonveg_ub_con = Constraint(mdl.T, rule=nonveg_ub_expr)

def hardness_lb_expr(mdl, t):
    return sum(mdl.h[o] * mdl.u[t,o] for o in mdl.O) >= 3 * mdl.f[t]

mdl.hardness_lb_con = Constraint(mdl.T, rule=hardness_lb_expr)

def hardness_ub_expr(mdl, t):
    return sum(mdl.h[o] * mdl.u[t,o] for o in mdl.O) <= 6 * mdl.f[t]

mdl.hardness_ub_con = Constraint(mdl.T, rule=hardness_ub_expr)

## Solve

In [28]:
data = {None: {
    'O': {None: ['VEG1', 'VEG2', 'OIL1', 'OIL2', 'OIL3']},
    'T': {None: [1, 2, 3, 4, 5, 6]},
    'h': {'VEG1': 8.8, 'VEG2': 6.1, 'OIL1': 2, 'OIL2': 4.2, 'OIL3': 5},
    'g': {(1,'VEG1'): 110, (2,'VEG1'): 130, (3,'VEG1'): 110, (4,'VEG1'): 120, (5,'VEG1'): 100, (6,'VEG1'): 90,
          (1,'VEG2'): 120, (2,'VEG2'): 130, (3,'VEG2'): 140, (4,'VEG2'): 110, (5,'VEG2'): 120, (6,'VEG2'): 100,
          (1,'OIL1'): 130, (2,'OIL1'): 110, (3,'OIL1'): 130, (4,'OIL1'): 120, (5,'OIL1'): 150, (6,'OIL1'): 140,
          (1,'OIL2'): 110, (2,'OIL2'): 90, (3,'OIL2'): 100, (4,'OIL2'): 120, (5,'OIL2'): 110, (6,'OIL2'): 80,
          (1,'OIL3'): 115, (2,'OIL3'): 115, (3,'OIL3'): 95, (4,'OIL3'): 125, (5,'OIL3'): 105, (6,'OIL3'): 135},
    'p': {None: 150},
    'q': {None: 5}
}}

In [29]:
instance = mdl.create_instance(data)
opt = SolverFactory('glpk')
foodManufacture1_results = opt.solve(instance, tee=True)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /tmp/tmp1i8tfukf.glpk.raw --wglp /tmp/tmpzsdxy804.glpk.glp --cpxlp
 /tmp/tmpi_sa7iy8.pyomo.lp
Reading problem data from '/tmp/tmpi_sa7iy8.pyomo.lp'...
66 rows, 97 columns, 259 non-zeros
627 lines were read
Writing problem data to '/tmp/tmpzsdxy804.glpk.glp'...
556 lines were written
GLPK Simplex Optimizer, v4.65
66 rows, 97 columns, 259 non-zeros
Preprocessing...
60 rows, 61 columns, 218 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  8.800e+00  ratio =  8.800e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 60
      0: obj =   7.500000000e+03 inf =   2.500e+03 (5)
     38: obj =   2.707592593e+04 inf =   0.000e+00 (0)
*    54: obj =   1.078425926e+05 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.1 Mb (136997 bytes)
Writing basic solution to '/tmp/tmp1i8tfukf.glpk.raw'...
172 lines were written


## Result

Variables

In [30]:
for v in instance.component_objects(Var, active=True):
    print(v)
    for index in v:
        print(' ', index, value(v[index]))

b
  (1, 'VEG1') 0.0
  (1, 'VEG2') 0.0
  (1, 'OIL1') 0.0
  (1, 'OIL2') 0.0
  (1, 'OIL3') 0.0
  (2, 'VEG1') 0.0
  (2, 'VEG2') 0.0
  (2, 'OIL1') 0.0
  (2, 'OIL2') 250.0
  (2, 'OIL3') 0.0
  (3, 'VEG1') 0.0
  (3, 'VEG2') 0.0
  (3, 'OIL1') 0.0
  (3, 'OIL2') 0.0
  (3, 'OIL3') 0.0
  (4, 'VEG1') 0.0
  (4, 'VEG2') 0.0
  (4, 'OIL1') 0.0
  (4, 'OIL2') 0.0
  (4, 'OIL3') 0.0
  (5, 'VEG1') 0.0
  (5, 'VEG2') 0.0
  (5, 'OIL1') 0.0
  (5, 'OIL2') 0.0
  (5, 'OIL3') 499.999999999997
  (6, 'VEG1') 659.259259259259
  (6, 'VEG2') 540.74074074074
  (6, 'OIL1') 0.0
  (6, 'OIL2') 750.0
  (6, 'OIL3') 0.0
u
  (1, 'VEG1') 11.1111111111093
  (1, 'VEG2') 188.888888888892
  (1, 'OIL1') 0.0
  (1, 'OIL2') 250.0
  (1, 'OIL3') 0.0
  (2, 'VEG1') 85.1851851851852
  (2, 'VEG2') 114.814814814815
  (2, 'OIL1') 0.0
  (2, 'OIL2') 0.0
  (2, 'OIL3') 250.0
  (3, 'VEG1') 85.1851851851853
  (3, 'VEG2') 114.814814814814
  (3, 'OIL1') 0.0
  (3, 'OIL2') 0.0
  (3, 'OIL3') 250.0
  (4, 'VEG1') 159.259259259258
  (4, 'VEG2') 40.740740740743

Parameters

In [31]:
for p in instance.component_objects(Param, active=True):
    print(p.name)
    for index in p:
        print(' ', index,p[index])

h
  VEG1 8.8
  VEG2 6.1
  OIL1 2
  OIL2 4.2
  OIL3 5
g
  (1, 'VEG1') 110
  (2, 'VEG1') 130
  (3, 'VEG1') 110
  (4, 'VEG1') 120
  (5, 'VEG1') 100
  (6, 'VEG1') 90
  (1, 'VEG2') 120
  (2, 'VEG2') 130
  (3, 'VEG2') 140
  (4, 'VEG2') 110
  (5, 'VEG2') 120
  (6, 'VEG2') 100
  (1, 'OIL1') 130
  (2, 'OIL1') 110
  (3, 'OIL1') 130
  (4, 'OIL1') 120
  (5, 'OIL1') 150
  (6, 'OIL1') 140
  (1, 'OIL2') 110
  (2, 'OIL2') 90
  (3, 'OIL2') 100
  (4, 'OIL2') 120
  (5, 'OIL2') 110
  (6, 'OIL2') 80
  (1, 'OIL3') 115
  (2, 'OIL3') 115
  (3, 'OIL3') 95
  (4, 'OIL3') 125
  (5, 'OIL3') 105
  (6, 'OIL3') 135
p
  None p
q
  None q


# Food Manufacture 2

In [32]:
mdl = AbstractModel()

## Sets

$O$ Oils

$T$ Time periods

In [33]:
mdl.O = Set()
mdl.T = Set()

## Parameters

$h_o$ Hardness of oil o

$g_{t,o}$ Purchase price of oil o in period t

$p$ Unit sales price of final product

$q$ Storage cost per product and period

In [34]:
mdl.h = Param(mdl.O)
mdl.g = Param(mdl.T, mdl.O)
mdl.p = Param(within=NonNegativeReals)
mdl.q = Param(within=NonNegativeReals)

## Variables

$b_{t,o}$ Oil o bought in period t

$u_{t,o}$ Oil o used in period t

$s_{t,o}$ Oil o stored in period t

$f_t$ Food produced in period t

$d_{t,o}$ Indicator variable for oil o in period t

$b_{t,o},u_{t,o},s_{t,o}\geq 0\:\forall t\in T,\forall o\in O$

$f_t\geq 0\:\forall t\in T$

$d_{t,o}\in\{0,1\}\:\forall t\in T, \forall o\in O$

In [35]:
mdl.b = Var(mdl.T, mdl.O, domain=NonNegativeReals)
mdl.u = Var(mdl.T, mdl.O, domain=NonNegativeReals)
mdl.s = Var(mdl.T, mdl.O, bounds=(0, 1000), domain=NonNegativeReals)
mdl.f = Var(mdl.T, domain=NonNegativeReals)
mdl.d = Var(mdl.T, mdl.O, domain=Binary)

## Objective Function

$\max\sum\limits_{t\in T}(p*f_t)-\sum\limits_{t\in T}\sum\limits_{o\in O}(g_{t,o}*b_{t,o}+q*s_{t,o})$

In [36]:
def max_profit(mdl):
    Revenue = sum(mdl.p * mdl.f[t] for t in mdl.T)
    RawCost = sum(mdl.g[t, o] * mdl.b[t, o] for t in mdl.T for o in mdl.O)
    StorageCost = sum(mdl.q * mdl.s[t, o] for t in mdl.T for o in mdl.O)
    return Revenue - RawCost - StorageCost

mdl.OBJ = Objective(rule=max_profit, sense=maximize)

## Constraints

**balance**

$\sum\limits_{o\in O}u_{t,o}=f_t\:\forall t\in T$

$500+b_{t_0,o}=u_{t_0}+s_{t_0,o}\:\forall o\in O$

$s_{t-1,o}+b_{t,o}=u_{t,o}+s_{t,o}\:\forall t\in T\setminus{t_0}, \forall o\in O$

$s_{t_e,o}=500\:\forall o\in O$

**production**

$\sum\limits_{o\in V}u_{t,o}\leq 200\:\forall t\in T$

$\sum\limits_{o\in N}u_{t,o}\leq 250\:\forall t\in T$

**quality**

$3*f_t\leq\sum\limits_{o\in O}h_o*u_{t,o}\leq 6*f_t$

**logical**

$u_{t,o}-20*d_{t,o}\geq 0\:\forall t\in T, \forall o\in O$

$u_{t,o}-200*d_{t,o}\leq 0\:\forall t\in T, \forall o\in V$

$u_{t,o}-250*d_{t,o}\leq 0\:\forall t\in T, \forall o\in N$

$\sum\limits_{o\in O}d_{t,o}\leq 3\:\forall t\in T$

$d_{t,o_0}-d_{t,o_1}-2*d_{t,o_4}\leq 0\:\forall t\in T$

In [37]:
def material_balance_expr(mdl, t):
    return sum(mdl.u[t, o] for o in mdl.O) == mdl.f[t]

mdl.material_balance_con = Constraint(mdl.T, rule=material_balance_expr)

def initial_balance_expr(mdl, o):
    return 500 + mdl.b[1, o] == mdl.u[1, o] + mdl.s[1, o]

mdl.initial_balance_con = Constraint(mdl.O, rule=initial_balance_expr)

def flow_balance_expr(mdl, t, o):
    if t >= 2:
        return mdl.s[t-1, o] + mdl.b[t, o] == mdl.u[t, o] + mdl.s[t, o]
    return Constraint.Skip

mdl.flow_balance_con = Constraint(mdl.T, mdl.O, rule=flow_balance_expr)

def end_balance_expr(mdl, o):
    return mdl.s[6, o] == 500

mdl.end_balance_con = Constraint(mdl.O, rule=end_balance_expr)

def veg_ub_expr(mdl, t):
    return sum(mdl.u[t, o] for o in mdl.O if 'VEG' in o) <= 200

mdl.veg_ub_con = Constraint(mdl.T, rule=veg_ub_expr)

def nonveg_ub_expr(mdl, t):
    return sum(mdl.u[t, o] for o in mdl.O if 'OIL' in o) <= 250

mdl.nonveg_ub_con = Constraint(mdl.T, rule=nonveg_ub_expr)

def hardness_lb_expr(mdl, t):
    return sum(mdl.h[o] * mdl.u[t,o] for o in mdl.O) >= 3 * mdl.f[t]

mdl.hardness_lb_con = Constraint(mdl.T, rule=hardness_lb_expr)

def hardness_ub_expr(mdl, t):
    return sum(mdl.h[o] * mdl.u[t,o] for o in mdl.O) <= 6 * mdl.f[t]

mdl.hardness_ub_con = Constraint(mdl.T, rule=hardness_ub_expr)

def cond1_lb_expr(mdl, t, o):
    return mdl.u[t, o] - 20 * mdl.d[t, o] >= 0

mdl.cond1_lb_con = Constraint(mdl.T, mdl.O, rule=cond1_lb_expr)

def cond1_ub_expr(mdl, t, o):
    if o in ['VEG1', 'VEG2']:
        return mdl.u[t, o] - 200 * mdl.d[t, o] <= 0
    elif o in ['OIL1', 'OIL2', 'OIL3']:
        return mdl.u[t, o] - 250 * mdl.d[t, o] <= 0
    return Constraint.Skip

mdl.cond1_ub_con = Constraint(mdl.T, mdl.O, rule=cond1_ub_expr)

def cond2_expr(mdl, t):
    return sum(mdl.d[t, o] for o in mdl.O) <= 3

mdl.cond2_con = Constraint(mdl.T, rule=cond2_expr)

def cond3_expr(mdl, t):
    return mdl.d[t, 'VEG1'] + mdl.d[t, 'VEG2'] - 2 * mdl.d[t, 'OIL3'] <= 0

mdl.cond3_con = Constraint(mdl.T, rule=cond3_expr)

## Solve

In [38]:
data = {None: {
    'O': {None: ['VEG1', 'VEG2', 'OIL1', 'OIL2', 'OIL3']},
    'T': {None: [1, 2, 3, 4, 5, 6]},
    'h': {'VEG1': 8.8, 'VEG2': 6.1, 'OIL1': 2, 'OIL2': 4.2, 'OIL3': 5},
    'g': {(1,'VEG1'): 110, (2,'VEG1'): 130, (3,'VEG1'): 110, (4,'VEG1'): 120, (5,'VEG1'): 100, (6,'VEG1'): 90,
          (1,'VEG2'): 120, (2,'VEG2'): 130, (3,'VEG2'): 140, (4,'VEG2'): 110, (5,'VEG2'): 120, (6,'VEG2'): 100,
          (1,'OIL1'): 130, (2,'OIL1'): 110, (3,'OIL1'): 130, (4,'OIL1'): 120, (5,'OIL1'): 150, (6,'OIL1'): 140,
          (1,'OIL2'): 110, (2,'OIL2'): 90, (3,'OIL2'): 100, (4,'OIL2'): 120, (5,'OIL2'): 110, (6,'OIL2'): 80,
          (1,'OIL3'): 115, (2,'OIL3'): 115, (3,'OIL3'): 95, (4,'OIL3'): 125, (5,'OIL3'): 105, (6,'OIL3'): 135},
    'p': {None: 150},
    'q': {None: 5}
}}

In [39]:
instance = mdl.create_instance(data)
opt = SolverFactory('glpk')
foodManufacture1_results = opt.solve(instance, tee=True)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /tmp/tmpcgzri4o4.glpk.raw --wglp /tmp/tmp8tvfj7jh.glpk.glp --cpxlp
 /tmp/tmpbk65c_eg.pyomo.lp
Reading problem data from '/tmp/tmpbk65c_eg.pyomo.lp'...
/tmp/tmpbk65c_eg.pyomo.lp:1042: warning: lower bound of variable 'x97' redefined
/tmp/tmpbk65c_eg.pyomo.lp:1042: warning: upper bound of variable 'x97' redefined
138 rows, 127 columns, 427 non-zeros
30 integer variables, all of which are binary
1072 lines were read
Writing problem data to '/tmp/tmp8tvfj7jh.glpk.glp'...
965 lines were written
GLPK Integer Optimizer, v4.65
138 rows, 127 columns, 427 non-zeros
30 integer variables, all of which are binary
Preprocessing...
132 rows, 91 columns, 386 non-zeros
30 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  2.500e+02  ratio =  2.500e+02
GM: min|aij| =  5.318e-01  max|aij| =  1.880e+00  ratio =  3.536e+00
EQ: min|aij| =  2.828e-01  max|aij| =  1.000e+00  ratio =

## Result

Variables

In [40]:
for v in instance.component_objects(Var, active=True):
    print(v)
    for index in v:
        print(' ', index, value(v[index]))

b
  (1, 'VEG1') 0.0
  (1, 'VEG2') 0.0
  (1, 'OIL1') 0.0
  (1, 'OIL2') 0.0
  (1, 'OIL3') 0.0
  (2, 'VEG1') -2.8421709430404e-14
  (2, 'VEG2') 0.0
  (2, 'OIL1') 0.0
  (2, 'OIL2') 0.0
  (2, 'OIL3') 0.0
  (3, 'VEG1') 5.6843418860808e-14
  (3, 'VEG2') 0.0
  (3, 'OIL1') 0.0
  (3, 'OIL2') 0.0
  (3, 'OIL3') 770.0
  (4, 'VEG1') -2.8421709430404e-14
  (4, 'VEG2') 0.0
  (4, 'OIL1') 0.0
  (4, 'OIL2') 0.0
  (4, 'OIL3') 0.0
  (5, 'VEG1') 0.0
  (5, 'VEG2') 0.0
  (5, 'OIL1') 0.0
  (5, 'OIL2') 0.0
  (5, 'OIL3') 0.0
  (6, 'VEG1') 480.37037037037
  (6, 'VEG2') 629.62962962963
  (6, 'OIL1') 0.0
  (6, 'OIL2') 730.0
  (6, 'OIL3') 0.0
u
  (1, 'VEG1') 0.0
  (1, 'VEG2') 200.0
  (1, 'OIL1') 0.0
  (1, 'OIL2') 40.0
  (1, 'OIL3') 210.0
  (2, 'VEG1') 85.1851851851852
  (2, 'VEG2') 114.814814814815
  (2, 'OIL1') 0.0
  (2, 'OIL2') 0.0
  (2, 'OIL3') 250.0
  (3, 'VEG1') 85.1851851851852
  (3, 'VEG2') 114.814814814815
  (3, 'OIL1') 0.0
  (3, 'OIL2') 0.0
  (3, 'OIL3') 250.0
  (4, 'VEG1') 155.0
  (4, 'VEG2') 0.0
  (4, 'OI

Parameters

In [41]:
for p in instance.component_objects(Param, active=True):
    print(p.name)
    for index in p:
        print(' ', index,p[index])

h
  VEG1 8.8
  VEG2 6.1
  OIL1 2
  OIL2 4.2
  OIL3 5
g
  (1, 'VEG1') 110
  (2, 'VEG1') 130
  (3, 'VEG1') 110
  (4, 'VEG1') 120
  (5, 'VEG1') 100
  (6, 'VEG1') 90
  (1, 'VEG2') 120
  (2, 'VEG2') 130
  (3, 'VEG2') 140
  (4, 'VEG2') 110
  (5, 'VEG2') 120
  (6, 'VEG2') 100
  (1, 'OIL1') 130
  (2, 'OIL1') 110
  (3, 'OIL1') 130
  (4, 'OIL1') 120
  (5, 'OIL1') 150
  (6, 'OIL1') 140
  (1, 'OIL2') 110
  (2, 'OIL2') 90
  (3, 'OIL2') 100
  (4, 'OIL2') 120
  (5, 'OIL2') 110
  (6, 'OIL2') 80
  (1, 'OIL3') 115
  (2, 'OIL3') 115
  (3, 'OIL3') 95
  (4, 'OIL3') 125
  (5, 'OIL3') 105
  (6, 'OIL3') 135
p
  None p
q
  None q


# Factory Planning 1

In [52]:
mdl = AbstractModel()

## Sets

$T$ Time periods

$P$ Products

$M$ Machines

In [53]:
mdl.T = Set()
mdl.P = Set()
mdl.M = Set()

## Parameters

$f_{p,m}$ Time used to manufacture product p on machine m (in hours)

$l_{t,p}$ Upper sales limit on product p in period t

$k_{p}$ Profit for product p

$q_{t,m}$ Number of broken machines m in period t

$c_{m}$ Number of machines

$g$ Hours each machine can work

$z$ Maximum stock of each product in last period

$r$ Storage cost per product per month

In [54]:
mdl.f = Param(mdl.P, mdl.M)
mdl.l = Param(mdl.T, mdl.P)
mdl.k = Param(mdl.P)
mdl.q = Param(mdl.T, mdl.M)
mdl.c = Param(mdl.M)
mdl.g = Param(within=NonNegativeReals)
mdl.z = Param(within=NonNegativeReals)
mdl.r = Param(within=NonNegativeReals)

## Variables

$b_{t,p}$ Amount of product p produced in period t

$u_{t,p}$ Amount of product p sold in period t

$s_{t,p}$ Amount of product p sold in period t

$b_{t,p},u_{t,p},s_{t,p}\geq 0\:\forall t\in T, \forall p\in P$

In [55]:
mdl.b = Var(mdl.T, mdl.P, domain=NonNegativeReals)
mdl.u = Var(mdl.T, mdl.P, domain=NonNegativeReals)
mdl.s = Var(mdl.T, mdl.P, bounds=(0, 100), domain=NonNegativeReals)

## Objective Function

$\max\sum\limits_{t\in T}\sum\limits_{p\in P}k_p*u_{t,p}-r*s_{t,p}$

In [56]:
def max_profit(mdl):
    return sum(mdl.k[p] * mdl.u[t, p] - mdl.r * mdl.s[t, p] for t in mdl.T for p in mdl.P)

mdl.OBJ = Objective(rule=max_profit, sense=maximize)

## Constraints

**Balance**

$b_{t_0,p}=u_{t_0,p}+s_{t_0,p}\:\forall p\in P$

$s_{t-1,p}+b_{t,p}=u_{t,p}+s_{t,p}\:\forall t\in T \setminus{t_0}, \forall p\in P$

$s_{t_e}=z\:\forall p\in P$

**Sales**

$u_{t,p}\leq l_{t,p}\:\forall t\in T, \forall p\in P$

**Machine**

$\sum\limits_{p\in P}f_{p,m}*b_{t,p}\leq g*(c_{m}-q_{t,m})\:\forall t\in T, \forall m\in M$

In [57]:
def initial_balance_expr(mdl, p):
    return mdl.b[1, p] == mdl.u[1, p] + mdl.s[1, p]

mdl.initial_balance_con = Constraint(mdl.P, rule=initial_balance_expr)

def balance_expr(mdl, t, p):
    if t >= 2:
        return mdl.s[t-1, p] + mdl.b[t, p] == mdl.u[t, p] + mdl.s[t, p]
    return Constraint.Skip

mdl.balance_con = Constraint(mdl.T, mdl.P, rule=balance_expr)

def end_balance_expr(mdl, p):
    return mdl.s[6, p] == mdl.z

mdl.end_balance = Constraint(mdl.P, rule=end_balance_expr)

def sales_ub_expr(mdl, t, p):
    return mdl.u[t, p] <= mdl.l[t, p]

mdl.sales_ub_con = Constraint(mdl.T, mdl.P, rule=sales_ub_expr)

def machine_availability_expr(mdl, t, m):
        return sum(mdl.f[p, m] * mdl.b[t, p] for p in mdl.P) <= mdl.g * (mdl.c[m] - mdl.q[t, m])

mdl.machine_availability_con = Constraint(mdl.T, mdl.M, rule=machine_availability_expr)

## Solve

In [69]:
n_shifts = 2
n_work_hours = 8
n_work_days = 6*4
g = n_shifts * n_work_hours * n_work_days

data = {None: {
    'T': {None: [1, 2, 3, 4, 5, 6]},
    'P': {None: ['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7']},
    'M': {None: ['GRINDING', 'VDRILL', 'HDRILL', 'BORING', 'PLANING']},
    'f': {('P1','GRINDING'): 0.5, ('P1','VDRILL'): 0.1, ('P1','HDRILL'): 0.2, ('P1','BORING'): 0.05, ('P1','PLANING'): 0, 
          ('P2','GRINDING'): 0.7, ('P2','VDRILL'): 0.2, ('P2','HDRILL'): 0, ('P2','BORING'): 0.03, ('P2','PLANING'): 0, 
          ('P3','GRINDING'): 0, ('P3','VDRILL'): 0, ('P3','HDRILL'): 0.8, ('P3','BORING'): 0, ('P3','PLANING'): 0.01,
          ('P4','GRINDING'): 0, ('P4','VDRILL'): 0.3, ('P4','HDRILL'): 0, ('P4','BORING'): 0.07, ('P4','PLANING'): 0, 
          ('P5','GRINDING'): 0.3, ('P5','VDRILL'): 0, ('P5','HDRILL'): 0, ('P5','BORING'): 0.1, ('P5','PLANING'): 0.05, 
          ('P6','GRINDING'): 0.2, ('P6','VDRILL'): 0.6, ('P6','HDRILL'): 0, ('P6','BORING'): 0, ('P6','PLANING'): 0, 
          ('P7','GRINDING'): 0.5, ('P7','VDRILL'): 0, ('P7','HDRILL'): 0.6, ('P7','BORING'): 0.08, ('P7','PLANING'): 0.05},
    'l': {(1,'P1'): 500, (1,'P2'): 1000, (1,'P3'): 300, (1,'P4'): 300, (1,'P5'): 800, (1,'P6'): 200, (1,'P7'): 100, 
          (2,'P1'): 600, (2,'P2'): 500, (2,'P3'): 200, (2,'P4'): 0, (2,'P5'): 400, (2,'P6'): 300, (2,'P7'): 150,
          (3,'P1'): 300, (3,'P2'): 600, (3,'P3'): 0, (3,'P4'): 0, (3,'P5'): 500, (3,'P6'): 400, (3,'P7'): 100,
          (4,'P1'): 200, (4,'P2'): 300, (4,'P3'): 400, (4,'P4'): 500, (4,'P5'): 200, (4,'P6'): 0, (4,'P7'): 100,
          (5,'P1'): 0, (5,'P2'): 100, (5,'P3'): 500, (5,'P4'): 100, (5,'P5'): 1000, (5,'P6'): 300, (5,'P7'): 0,
          (6,'P1'): 500, (6,'P2'): 500, (6,'P3'): 100, (6,'P4'): 300, (6,'P5'): 1100, (6,'P6'): 500, (6,'P7'): 60},
    'k': {'P1': 10, 'P2': 6, 'P3': 8, 'P4': 4, 'P5': 11, 'P6': 9, 'P7': 3},
    'q': {(1,'GRINDING'): 1, (1,'VDRILL'): 0, (1,'HDRILL'): 0, (1,'BORING'): 0, (1,'PLANING'): 0,
          (2,'GRINDING'): 0, (2,'VDRILL'): 0, (2,'HDRILL'): 2, (2,'BORING'): 0, (2,'PLANING'): 0,
          (3,'GRINDING'): 0, (3,'VDRILL'): 0, (3,'HDRILL'): 0, (3,'BORING'): 1, (3,'PLANING'): 0,
          (4,'GRINDING'): 0, (4,'VDRILL'): 1, (4,'HDRILL'): 0, (4,'BORING'): 0, (4,'PLANING'): 0,
          (5,'GRINDING'): 1, (5,'VDRILL'): 1, (5,'HDRILL'): 0, (5,'BORING'): 0, (5,'PLANING'): 0,
          (6,'GRINDING'): 0, (6,'VDRILL'): 0, (6,'HDRILL'): 1, (6,'BORING'): 0, (6,'PLANING'): 1},
    'c': {'GRINDING': 4, 'VDRILL': 2, 'HDRILL': 3, 'BORING': 1, 'PLANING': 1},
    'g': {None: g},
    'z': {None: 50},
    'r': {None: 0.5}
}}

In [70]:
instance = mdl.create_instance(data)
opt = SolverFactory('glpk')
factoryPlanning1_results = opt.solve(instance, tee=True)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /tmp/tmp7mr7f6iw.glpk.raw --wglp /tmp/tmpp2qz2p11.glpk.glp --cpxlp
 /tmp/tmp1yjnixf8.pyomo.lp
Reading problem data from '/tmp/tmp1yjnixf8.pyomo.lp'...
122 rows, 127 columns, 331 non-zeros
915 lines were read
Writing problem data to '/tmp/tmpp2qz2p11.glpk.glp'...
789 lines were written
GLPK Simplex Optimizer, v4.65
122 rows, 127 columns, 331 non-zeros
Preprocessing...
62 rows, 67 columns, 189 non-zeros
Scaling...
 A: min|aij| =  1.000e-02  max|aij| =  1.000e+00  ratio =  1.000e+02
GM: min|aij| =  5.081e-01  max|aij| =  1.968e+00  ratio =  3.873e+00
EQ: min|aij| =  2.582e-01  max|aij| =  1.000e+00  ratio =  3.873e+00
Constructing initial basis...
Size of triangular part is 62
      0: obj =  -2.775000000e+03 inf =   3.968e+02 (7)
     21: obj =  -3.750000000e+02 inf =   0.000e+00 (0)
*    79: obj =   9.371517857e+04 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.1 

## Result

Variables

In [71]:
for v in instance.component_objects(Var, active=True):
    print(v)
    for index in v:
        print(' ', index, value(v[index]))

b
  (1, 'P1') 500.0
  (1, 'P2') 888.571428571428
  (1, 'P3') 382.5
  (1, 'P4') 300.0
  (1, 'P5') 800.0
  (1, 'P6') 200.0
  (1, 'P7') 0.0
  (2, 'P1') 700.0
  (2, 'P2') 600.0
  (2, 'P3') 117.5
  (2, 'P4') 0.0
  (2, 'P5') 500.0
  (2, 'P6') 300.0
  (2, 'P7') 250.0
  (3, 'P1') 0.0
  (3, 'P2') 0.0
  (3, 'P3') 0.0
  (3, 'P4') 0.0
  (3, 'P5') 0.0
  (3, 'P6') 400.0
  (3, 'P7') 0.0
  (4, 'P1') 200.0
  (4, 'P2') 300.0
  (4, 'P3') 400.0
  (4, 'P4') 500.0
  (4, 'P5') 200.0
  (4, 'P6') 0.0
  (4, 'P7') 100.0
  (5, 'P1') 0.0
  (5, 'P2') 99.9999999999999
  (5, 'P3') 600.0
  (5, 'P4') 100.0
  (5, 'P5') 1100.0
  (5, 'P6') 300.0
  (5, 'P7') 100.0
  (6, 'P1') 550.0
  (6, 'P2') 550.0
  (6, 'P3') 0.0
  (6, 'P4') 350.0
  (6, 'P5') 0.0
  (6, 'P6') 550.0
  (6, 'P7') 0.0
u
  (1, 'P1') 500.0
  (1, 'P2') 888.571428571428
  (1, 'P3') 300.0
  (1, 'P4') 300.0
  (1, 'P5') 800.0
  (1, 'P6') 200.0
  (1, 'P7') -0.0
  (2, 'P1') 600.0
  (2, 'P2') 500.0
  (2, 'P3') 200.0
  (2, 'P4') 0.0
  (2, 'P5') 400.0
  (2, 'P6') 300.0
 

Parameter

In [72]:
for p in instance.component_objects(Param, active=True):
    print(p.name)
    for index in p:
        print(' ', index,p[index])

f
  ('P1', 'GRINDING') 0.5
  ('P1', 'VDRILL') 0.1
  ('P1', 'HDRILL') 0.2
  ('P1', 'BORING') 0.05
  ('P1', 'PLANING') 0
  ('P2', 'GRINDING') 0.7
  ('P2', 'VDRILL') 0.2
  ('P2', 'HDRILL') 0
  ('P2', 'BORING') 0.03
  ('P2', 'PLANING') 0
  ('P3', 'GRINDING') 0
  ('P3', 'VDRILL') 0
  ('P3', 'HDRILL') 0.8
  ('P3', 'BORING') 0
  ('P3', 'PLANING') 0.01
  ('P4', 'GRINDING') 0
  ('P4', 'VDRILL') 0.3
  ('P4', 'HDRILL') 0
  ('P4', 'BORING') 0.07
  ('P4', 'PLANING') 0
  ('P5', 'GRINDING') 0.3
  ('P5', 'VDRILL') 0
  ('P5', 'HDRILL') 0
  ('P5', 'BORING') 0.1
  ('P5', 'PLANING') 0.05
  ('P6', 'GRINDING') 0.2
  ('P6', 'VDRILL') 0.6
  ('P6', 'HDRILL') 0
  ('P6', 'BORING') 0
  ('P6', 'PLANING') 0
  ('P7', 'GRINDING') 0.5
  ('P7', 'VDRILL') 0
  ('P7', 'HDRILL') 0.6
  ('P7', 'BORING') 0.08
  ('P7', 'PLANING') 0.05
l
  (1, 'P1') 500
  (1, 'P2') 1000
  (1, 'P3') 300
  (1, 'P4') 300
  (1, 'P5') 800
  (1, 'P6') 200
  (1, 'P7') 100
  (2, 'P1') 600
  (2, 'P2') 500
  (2, 'P3') 200
  (2, 'P4') 0
  (2, 'P5') 400
  

# Factory Planning 2

In [73]:
mdl = AbstractModel()

## Sets

$T$ Time periods

$P$ Products

$M$ Machines

In [74]:
mdl.T = Set()
mdl.P = Set()
mdl.M = Set()

## Parameters

$f_{p,m}$ Time used to manufacture product p on machine m (in hours)

$l_{t,p}$ Upper sales limit on product p in period t

$k_{p}$ Profit for product p

$c_{m}$ Number of machines

$v_{m}$ Number of machine m that need to be down in a month

$g$ Hours each machine can work

$z$ Maximum stock of each product in last period

$r$ Storage cost per product per month

In [75]:
mdl.f = Param(mdl.P, mdl.M)
mdl.l = Param(mdl.T, mdl.P)
mdl.k = Param(mdl.P)
mdl.c = Param(mdl.M)
mdl.v = Param(mdl.M)
mdl.g = Param(within=NonNegativeReals)
mdl.z = Param(within=NonNegativeReals)
mdl.r = Param(within=NonNegativeReals)

## Variables

$b_{t,p}$ Amount of product p produced in period t

$u_{t,p}$ Amount of product p sold in period t

$s_{t,p}$ Amount of product p sold in period t

$d_{t,m}$ Number of machines m down in period p

$b_{t,p},u_{t,p},s_{t,p}\geq 0\:\forall t\in T, \forall p\in P$

$0\leq d_{t,m}\leq k_m, d_{t,m}\in \mathbb{Z}\:\forall t\in T, \forall m\in M$

In [76]:
mdl.b = Var(mdl.T, mdl.P, domain=NonNegativeReals)
mdl.u = Var(mdl.T, mdl.P, domain=NonNegativeReals)
mdl.s = Var(mdl.T, mdl.P, bounds=(0, 100), domain=NonNegativeReals)
mdl.d = Var(mdl.T, mdl.M, domain=NonNegativeIntegers)

## Objective Function

$\max\sum\limits_{t\in T}\sum\limits_{p\in P}k_p*u_{t,p}-r*s_{t,p}$

In [77]:
def max_profit(mdl):
    return sum(mdl.k[p] * mdl.u[t, p] - mdl.r * mdl.s[t, p] for t in mdl.T for p in mdl.P)

mdl.OBJ = Objective(rule=max_profit, sense=maximize)

## Constraints

**Balance**

$b_{t_0,p}=u_{t_0,p}+s_{t_0,p}\:\forall p\in P$

$s_{t-1,p}+b_{t,p}=u_{t,p}+s_{t,p}\:\forall t\in T \setminus{t_0}, \forall p\in P$

$s_{t_e}=z\:\forall p\in P$

**Sales**

$u_{t,p}\leq l_{t,p}\:\forall t\in T, \forall p\in P$

**Machine**

$\sum\limits_{p\in P}f_{p,m}*b_{t,p}\leq g*(c_{m}-d_{t,m})\:\forall t\in T, \forall m\in M$

$\sum\limits_{t\in T}d_{t,m}=v_m$

In [78]:
def initial_balance_expr(mdl, p):
    return mdl.b[1, p] == mdl.u[1, p] + mdl.s[1, p]

mdl.initial_balance_con = Constraint(mdl.P, rule=initial_balance_expr)

def balance_expr(mdl, t, p):
    if t >= 2:
        return mdl.s[t-1, p] + mdl.b[t, p] == mdl.u[t, p] + mdl.s[t, p]
    return Constraint.Skip

mdl.balance_con = Constraint(mdl.T, mdl.P, rule=balance_expr)

def end_balance_expr(mdl, p):
    return mdl.s[6, p] == mdl.z

mdl.end_balance = Constraint(mdl.P, rule=end_balance_expr)

def sales_ub_expr(mdl, t, p):
    return mdl.u[t, p] <= mdl.l[t, p]

mdl.sales_ub_con = Constraint(mdl.T, mdl.P, rule=sales_ub_expr)

def machine_availability_expr(mdl, t, m):
    return sum(mdl.f[p, m] * mdl.b[t, p] for p in mdl.P) <= mdl.g * (mdl.c[m] - mdl.d[t, m])

mdl.machine_availability_con = Constraint(mdl.T, mdl.M, rule=machine_availability_expr)

def machine_maintenance_expr(mdl, t, m):
    return sum(mdl.d[t, m] for t in mdl.T) == mdl.v[m]

mdl.machine_maintenance_con = Constraint(mdl.T, mdl.M, rule=machine_maintenance_expr)

## Solve

In [79]:
n_shifts = 2
n_work_hours = 8
n_work_days = 6*4
g = n_shifts * n_work_hours * n_work_days

data = {None: {
    'T': {None: [1, 2, 3, 4, 5, 6]},
    'P': {None: ['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7']},
    'M': {None: ['GRINDING', 'VDRILL', 'HDRILL', 'BORING', 'PLANING']},
    'f': {('P1','GRINDING'): 0.5, ('P1','VDRILL'): 0.1, ('P1','HDRILL'): 0.2, ('P1','BORING'): 0.05, ('P1','PLANING'): 0, 
          ('P2','GRINDING'): 0.7, ('P2','VDRILL'): 0.2, ('P2','HDRILL'): 0, ('P2','BORING'): 0.03, ('P2','PLANING'): 0, 
          ('P3','GRINDING'): 0, ('P3','VDRILL'): 0, ('P3','HDRILL'): 0.8, ('P3','BORING'): 0, ('P3','PLANING'): 0.01,
          ('P4','GRINDING'): 0, ('P4','VDRILL'): 0.3, ('P4','HDRILL'): 0, ('P4','BORING'): 0.07, ('P4','PLANING'): 0, 
          ('P5','GRINDING'): 0.3, ('P5','VDRILL'): 0, ('P5','HDRILL'): 0, ('P5','BORING'): 0.1, ('P5','PLANING'): 0.05, 
          ('P6','GRINDING'): 0.2, ('P6','VDRILL'): 0.6, ('P6','HDRILL'): 0, ('P6','BORING'): 0, ('P6','PLANING'): 0, 
          ('P7','GRINDING'): 0.5, ('P7','VDRILL'): 0, ('P7','HDRILL'): 0.6, ('P7','BORING'): 0.08, ('P7','PLANING'): 0.05},
    'l': {(1,'P1'): 500, (1,'P2'): 1000, (1,'P3'): 300, (1,'P4'): 300, (1,'P5'): 800, (1,'P6'): 200, (1,'P7'): 100, 
          (2,'P1'): 600, (2,'P2'): 500, (2,'P3'): 200, (2,'P4'): 0, (2,'P5'): 400, (2,'P6'): 300, (2,'P7'): 150,
          (3,'P1'): 300, (3,'P2'): 600, (3,'P3'): 0, (3,'P4'): 0, (3,'P5'): 500, (3,'P6'): 400, (3,'P7'): 100,
          (4,'P1'): 200, (4,'P2'): 300, (4,'P3'): 400, (4,'P4'): 500, (4,'P5'): 200, (4,'P6'): 0, (4,'P7'): 100,
          (5,'P1'): 0, (5,'P2'): 100, (5,'P3'): 500, (5,'P4'): 100, (5,'P5'): 1000, (5,'P6'): 300, (5,'P7'): 0,
          (6,'P1'): 500, (6,'P2'): 500, (6,'P3'): 100, (6,'P4'): 300, (6,'P5'): 1100, (6,'P6'): 500, (6,'P7'): 60},
    'k': {'P1': 10, 'P2': 6, 'P3': 8, 'P4': 4, 'P5': 11, 'P6': 9, 'P7': 3},
    'c': {'GRINDING': 4, 'VDRILL': 2, 'HDRILL': 3, 'BORING': 1, 'PLANING': 1},
    'v': {'GRINDING': 2, 'VDRILL': 2, 'HDRILL': 3, 'BORING': 1, 'PLANING': 1},
    'g': {None: g},
    'z': {None: 50},
    'r': {None: 0.5}
}}

In [80]:
instance = mdl.create_instance(data)
opt = SolverFactory('glpk')
factoryPlanning2_results = opt.solve(instance, tee=True)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /tmp/tmph_nx3frz.glpk.raw --wglp /tmp/tmpwijokxyn.glpk.glp --cpxlp
 /tmp/tmp5obhuw24.pyomo.lp
Reading problem data from '/tmp/tmp5obhuw24.pyomo.lp'...
152 rows, 157 columns, 541 non-zeros
30 integer variables, none of which are binary
1276 lines were read
Writing problem data to '/tmp/tmpwijokxyn.glpk.glp'...
1204 lines were written
GLPK Integer Optimizer, v4.65
152 rows, 157 columns, 541 non-zeros
30 integer variables, none of which are binary
Preprocessing...
12 constraint coefficient(s) were reduced
100 rows, 107 columns, 430 non-zeros
30 integer variables, 12 of which are binary
Scaling...
 A: min|aij| =  1.000e-02  max|aij| =  3.840e+02  ratio =  3.840e+04
GM: min|aij| =  4.988e-01  max|aij| =  2.005e+00  ratio =  4.019e+00
EQ: min|aij| =  2.554e-01  max|aij| =  1.000e+00  ratio =  3.916e+00
2N: min|aij| =  2.000e-01  max|aij| =  1.600e+00  ratio =  8.000e+00
Constructing initial basis...
Size o

## Result

Variables

In [81]:
for v in instance.component_objects(Var, active=True):
    print(v)
    for index in v:
        print(' ', index, value(v[index]))

b
  (1, 'P1') 500.0
  (1, 'P2') 1000.0
  (1, 'P3') 300.0
  (1, 'P4') 300.0
  (1, 'P5') 800.0
  (1, 'P6') 200.0
  (1, 'P7') 100.0
  (2, 'P1') 600.0
  (2, 'P2') 500.0
  (2, 'P3') 200.0
  (2, 'P4') 0.0
  (2, 'P5') 400.0
  (2, 'P6') 300.0
  (2, 'P7') 150.0
  (3, 'P1') 400.0
  (3, 'P2') 700.0
  (3, 'P3') 100.0
  (3, 'P4') 100.0
  (3, 'P5') 600.0
  (3, 'P6') 400.0
  (3, 'P7') 200.0
  (4, 'P1') 0.0
  (4, 'P2') 0.0
  (4, 'P3') 0.0
  (4, 'P4') 0.0
  (4, 'P5') 0.0
  (4, 'P6') 0.0
  (4, 'P7') 0.0
  (5, 'P1') 0.0
  (5, 'P2') 100.0
  (5, 'P3') 500.0
  (5, 'P4') 100.0
  (5, 'P5') 1000.0
  (5, 'P6') 300.0
  (5, 'P7') 0.0
  (6, 'P1') 550.0
  (6, 'P2') 550.0
  (6, 'P3') 150.0
  (6, 'P4') 350.0
  (6, 'P5') 1150.0
  (6, 'P6') 550.0
  (6, 'P7') 110.0
u
  (1, 'P1') 500.0
  (1, 'P2') 1000.0
  (1, 'P3') 300.0
  (1, 'P4') 300.0
  (1, 'P5') 800.0
  (1, 'P6') 200.0
  (1, 'P7') 100.0
  (2, 'P1') 600.0
  (2, 'P2') 500.0
  (2, 'P3') 200.0
  (2, 'P4') 0.0
  (2, 'P5') 400.0
  (2, 'P6') 300.0
  (2, 'P7') 150.0
  (3, 

Parameters

In [82]:
for p in instance.component_objects(Param, active=True):
    print(p.name)
    for index in p:
        print(' ', index,p[index])

f
  ('P1', 'GRINDING') 0.5
  ('P1', 'VDRILL') 0.1
  ('P1', 'HDRILL') 0.2
  ('P1', 'BORING') 0.05
  ('P1', 'PLANING') 0
  ('P2', 'GRINDING') 0.7
  ('P2', 'VDRILL') 0.2
  ('P2', 'HDRILL') 0
  ('P2', 'BORING') 0.03
  ('P2', 'PLANING') 0
  ('P3', 'GRINDING') 0
  ('P3', 'VDRILL') 0
  ('P3', 'HDRILL') 0.8
  ('P3', 'BORING') 0
  ('P3', 'PLANING') 0.01
  ('P4', 'GRINDING') 0
  ('P4', 'VDRILL') 0.3
  ('P4', 'HDRILL') 0
  ('P4', 'BORING') 0.07
  ('P4', 'PLANING') 0
  ('P5', 'GRINDING') 0.3
  ('P5', 'VDRILL') 0
  ('P5', 'HDRILL') 0
  ('P5', 'BORING') 0.1
  ('P5', 'PLANING') 0.05
  ('P6', 'GRINDING') 0.2
  ('P6', 'VDRILL') 0.6
  ('P6', 'HDRILL') 0
  ('P6', 'BORING') 0
  ('P6', 'PLANING') 0
  ('P7', 'GRINDING') 0.5
  ('P7', 'VDRILL') 0
  ('P7', 'HDRILL') 0.6
  ('P7', 'BORING') 0.08
  ('P7', 'PLANING') 0.05
l
  (1, 'P1') 500
  (1, 'P2') 1000
  (1, 'P3') 300
  (1, 'P4') 300
  (1, 'P5') 800
  (1, 'P6') 200
  (1, 'P7') 100
  (2, 'P1') 600
  (2, 'P2') 500
  (2, 'P3') 200
  (2, 'P4') 0
  (2, 'P5') 400
  

# Manpower Planning

In [3]:
mdl = AbstractModel()

## Sets

$T$ Time periods

$S$ Workers

In [4]:
mdl.T = Set()
mdl.S = Set()

## Parameters

$r_{s,t}$ Manpower requirements

$q_{s}$ Percentage of workers with skill level s leaving in first year of service

$p_{s}$ Percentage of workers with skill level s leaving after first year of service

$o_{s}$ Number of workers with skill level s that can be recruited per year

$n_{s}$ Cost per superfluous worker with skill level s per year

$m_{s}$ Cost of short-term work per worker with skill level s per year

$l_{s}$ Cost of firing a worker with skill level s

$k_{s}$ Cost of retraining a worker with skill level s

In [5]:
mdl.r = Param(mdl.S, mdl.T)
mdl.q = Param(mdl.S)
mdl.p = Param(mdl.S)
mdl.o = Param(mdl.S)
mdl.n = Param(mdl.S)
mdl.m = Param(mdl.S)
mdl.l = Param(mdl.S)
mdl.k = Param(mdl.S)

## Variables

$u_{s,t}$ Number of employed workers with skill level s in period t

$v_{s,t}$ Number of recruited workers with skill level s in period t

$w_{s_i,s_j,t}$ Number of workers with level $s_i$ retrained to workers with level $s_j$ in period t, where $i<j$

$w_{s_j,s_i,t}$ Number of workers with level $s_i$ downgraded to workers with level $s_j$ in period t, where $i<j$

$x_{s,t}$ Number of workers with skill level s fired in period t

$y_{s,t}$ Number of workers with skill level s on short-term work in period t

$z_{s,t}$ Number of superfluous workers with skill level s in period t

In [6]:
mdl.u = Var(mdl.S, mdl.T, domain=NonNegativeReals)
mdl.v = Var(mdl.S, mdl.T, domain=NonNegativeReals)
mdl.w = Var(mdl.S, mdl.S, mdl.T, domain=NonNegativeReals)
mdl.x = Var(mdl.S, mdl.T, domain=NonNegativeReals)
mdl.y = Var(mdl.S, mdl.T, domain=NonNegativeReals)
mdl.z = Var(mdl.S, mdl.T, domain=NonNegativeReals)

## Objective Function

### Minimize Costs 

$\min\sum\limits_{t\in T \setminus{t_0}}k_{us}*w_{us,ss,t}+k_{ss}*w_{ss,sk,t}+\sum\limits_{s\in S}l_{s}*x_{s,t}+m_{s}*y_{s,t}+n_{s}*z_{s,t}$

retraining + fired + short-time + overmanning

In [7]:
def min_cost(mdl):
    retraining = sum(mdl.k['US']*mdl.w['US','SS',t] + mdl.k['SS']*mdl.w['SS','SK',t] for t in mdl.T if t > 1)
    redundancy = sum(mdl.l['US']*mdl.x['US',t] + mdl.l['SS']*mdl.x['SS',t] + mdl.l['SK']*mdl.x['SK',t] for t in mdl.T if t > 1)
    short_term = sum(mdl.m['US']*mdl.y['US',t] + mdl.m['SS']*mdl.y['SS',t] + mdl.m['SK']*mdl.y['SK',t] for t in mdl.T if t > 1)
    overmanning = sum(mdl.n['US']*mdl.z['US',t] + mdl.n['SS']*mdl.z['SS',t] + mdl.n['SK']*mdl.z['SK',t] for t in mdl.T if t > 1)
    return retraining + redundancy + short_term + overmanning
    
mdl.OBJ = Objective(rule=min_cost, sense=minimize)

### Minimize Redundancy

$\min\sum\limits_{s\in S}\sum\limits_{t\in T}x_{s,t}$

In [17]:
def min_redundancy(mdl):
    return sum(mdl.x[s,t] for s in mdl.S for t in mdl.T)

mdl.OBJ = Objective(rule=min_redundancy, sense=minimize)

## Constraints

**initial manpower**

$u_{s,t_0}=r_{s,t_0}$

**balance**

$u_{us,t}=(1-p_{us})*u_{us,t-1}+(1-q_{us})*v_{us,t}+0.5*(w_{ss,us,t}+w_{sk,us,t})-w_{us,ss,t}-x_{us,t}\:\forall t\in T \setminus{t_0}$

$u_{ss,t}=(1-p_{ss})*(u_{ss,t-1}+w_{us,ss,t})+(1-q_{ss})*v_{ss,t}+0.5*w_{sk,ss,t}-w_{ss,us,t}-w_{ss,sk,t}-x_{ss,t}\:\forall t\in T \setminus{t_0}$

$u_{sk,t}=(1-p_{sk})*(u_{sk,t-1}+w_{ss,sk,t})+(1-q_{sk})*v_{sk,t}-w_{sk,us,t}-w_{sk,ss,t}-x_{sk,t}\:\forall t\in T \setminus{t_0}$

**recruitment**

$v_{s,t}\leq o_s \:\forall s\in S, \forall t\in T \setminus{t_0}$

**retraining**

$w_{us,ss,t}\leq 200 \:\forall t\in T \setminus{t_0}$

$w_{ss,sk,t}-0.25*u_{sk,t}\leq 0 \:\forall t\in T \setminus{t_0}$

**overmanning**

$z_{s,t}\leq 150 \:\forall s\in S, \forall t\in T \setminus{t_0}$

**manpower requirements**

$u_{s,t}-z_{s,t}-0.5*y_{s,t} = r_{s,t} \:\forall s\in S, \forall t\in T \setminus{t_0}$

**short-time**

$y_{s,t}\leq 50 \:\forall s\in S, \forall t\in T \setminus{t_0}$

In [8]:
def initial_manpower_expr(mdl, s):
    return mdl.u[s, 1] == mdl.r[s, 1]

mdl.initial_manpower_con = Constraint(mdl.S, rule=initial_manpower_expr)

def balance_us_expr(mdl, t):
    if t > 1:
        return mdl.u['US',t] == (1-mdl.p['US'])*mdl.u['US',t-1]+(1-mdl.q['US'])*mdl.v['US',t]+0.5*(mdl.w['SS','US',t]+mdl.w['SK','US',t])-mdl.w['US','SS',t]-mdl.x['US',t]
    return Constraint.Skip

mdl.balance_us_con = Constraint(mdl.T, rule=balance_us_expr)

def balance_ss_expr(mdl, t):
    if t > 1:
        return mdl.u['SS',t] == (1-mdl.p['SS'])*(mdl.u['SS',t-1]+mdl.w['US','SS',t])+(1-mdl.q['SS'])*mdl.v['SS',t]+0.5*mdl.w['SK','SS',t]-mdl.w['SS','US',t]-mdl.w['SS','SK',t]-mdl.x['SS',t]
    return Constraint.Skip
    
mdl.balance_ss_con = Constraint(mdl.T, rule=balance_ss_expr)

def balance_sk_expr(mdl, t):
    if t > 1:
        return mdl.u['SK',t] == (1-mdl.p['SK'])*(mdl.u['SK',t-1]+mdl.w['SS','SK',t])+(1-mdl.q['SK'])*mdl.v['SK',t]-mdl.w['SK','US',t]-mdl.w['SK','SS',t]-mdl.x['SK',t]
    return Constraint.Skip
    
mdl.balance_sk_con = Constraint(mdl.T, rule=balance_sk_expr)

def recruitment_expr(mdl, s, t):
    if t > 1:
        return mdl.v[s,t] <= mdl.o[s]
    return Constraint.Skip

mdl.recruitment_con = Constraint(mdl.S, mdl.T, rule=recruitment_expr)

def retraining_us_expr(mdl, t):
    if t > 1:
        return mdl.w['US','SS',t] <= 200
    return Constraint.Skip

mdl.retraining_us_con = Constraint(mdl.T, rule=retraining_us_expr)

def retraining_ss_expr(mdl, t):
    if t > 1:
        return mdl.w['SS','SK',t]-0.25*mdl.u['SK',t] <= 0
    return Constraint.Skip

mdl.retraining_ss_con = Constraint(mdl.T, rule=retraining_ss_expr)

def overmanning_expr(mdl, t):
    if t > 1:
        return sum(mdl.z[s,t] for s in mdl.S) <= 150
    return Constraint.Skip

mdl.overmanning_con = Constraint(mdl.T, rule=overmanning_expr)

def manpower_requirements_expr(mdl, s, t):
    if t > 1:
        return mdl.u[s,t]-mdl.z[s,t]-0.5*mdl.y[s,t] == mdl.r[s,t]
    return Constraint.Skip

mdl.manpower_requirements_con = Constraint(mdl.S, mdl.T, rule=manpower_requirements_expr)

def short_time_expr(mdl, s, t):
    if t > 1:
        return mdl.y[s,t] <= 50
    return Constraint.Skip
    
mdl.short_time_con = Constraint(mdl.S, mdl.T, rule=short_time_expr)

## Solve

In [9]:
data = {None: {
    'T': {None: [1, 2, 3, 4]},
    'S': {None: ['US', 'SS', 'SK']},
    'r': {('US', 1): 2000, ('US',2): 1000, ('US',3): 500, ('US',4): 0,
          ('SS', 1): 1500, ('SS',2): 1400, ('SS',3): 2000, ('SS',4): 2500,
          ('SK', 1): 1000, ('SK',2): 1000, ('SK',3): 1500, ('SK',4): 2000},
    'q': {'US': 0.25, 'SS': 0.20, 'SK': 0.10},
    'p': {'US': 0.10, 'SS': 0.05, 'SK': 0.05},
    'o': {'US': 500, 'SS': 800, 'SK': 500},
    'l': {'US': 200, 'SS': 500, 'SK': 500},
    'n': {'US': 1500, 'SS': 2000, 'SK': 3000},
    'm': {'US': 500, 'SS': 400, 'SK': 400},
    'k': {'US': 400, 'SS': 500, 'SK': 0}
}}

In [10]:
instance = mdl.create_instance(data)
opt = SolverFactory('glpk')
manpowerPlanning_results = opt.solve(instance, tee=True)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /tmp/tmp76j1ak32.glpk.raw --wglp /tmp/tmpoer971x3.glpk.glp --cpxlp
 /tmp/tmpwdwzd5kw.pyomo.lp
Reading problem data from '/tmp/tmpwdwzd5kw.pyomo.lp'...
49 rows, 64 columns, 133 non-zeros
384 lines were read
Writing problem data to '/tmp/tmpoer971x3.glpk.glp'...
321 lines were written
GLPK Simplex Optimizer, v4.65
49 rows, 64 columns, 133 non-zeros
Preprocessing...
24 rows, 42 columns, 87 non-zeros
Scaling...
 A: min|aij| =  2.500e-01  max|aij| =  1.000e+00  ratio =  4.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 24
      0: obj =  -8.272500000e+06 inf =   1.190e+04 (8)
     20: obj =   5.559210526e+05 inf =   0.000e+00 (0)
*    25: obj =   4.986772853e+05 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.1 Mb (80836 bytes)
Writing basic solution to '/tmp/tmp76j1ak32.glpk.raw'...
122 lines were written


## Result

In [11]:
instance.pprint()

12 Set Declarations
    S : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'US', 'SS', 'SK'}
    T : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {1, 2, 3, 4}
    manpower_requirements_con_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    S*T :   12 : {('US', 1), ('US', 2), ('US', 3), ('US', 4), ('SS', 1), ('SS', 2), ('SS', 3), ('SS', 4), ('SK', 1), ('SK', 2), ('SK', 3), ('SK', 4)}
    r_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    S*T :   12 : {('US', 1), ('US', 2), ('US', 3), ('US', 4), ('SS', 1), ('SS', 2), ('SS', 3), ('SS', 4), ('SK', 1), ('SK', 2), ('SK', 3), ('SK', 4)}
    recruitment_con_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2

Variables

In [33]:
for v in instance.component_objects(Var, active=True):
    print(v)
    for index in v:
        print(' ', index, value(v[index]))

u
  ('US', 1) 2000.0
  ('US', 2) 1000.0
  ('US', 3) 500.0
  ('US', 4) 0.0
  ('SS', 1) 1500.0
  ('SS', 2) 1400.0
  ('SS', 3) 2000.0
  ('SS', 4) 2500.0
  ('SK', 1) 1000.0
  ('SK', 2) 1000.0
  ('SK', 3) 1500.0
  ('SK', 4) 2000.0
v
ERROR: evaluating object as numeric value: v[US,1]
        (object: <class 'pyomo.core.base.var._GeneralVarData'>)
    No value for uninitialized NumericValue object v[US,1]


ValueError: No value for uninitialized NumericValue object v[US,1]

In [30]:
list(list(instance.component_objects(Var, active=True))[1])

[('US', 1),
 ('US', 2),
 ('US', 3),
 ('US', 4),
 ('SS', 1),
 ('SS', 2),
 ('SS', 3),
 ('SS', 4),
 ('SK', 1),
 ('SK', 2),
 ('SK', 3),
 ('SK', 4)]

Parameters

In [14]:
for p in instance.component_objects(Param, active=True):
    print(p.name)
    for index in p:
        print(' ', index,p[index])

r
  ('US', 1) 2000
  ('US', 2) 1000
  ('US', 3) 500
  ('US', 4) 0
  ('SS', 1) 1500
  ('SS', 2) 1400
  ('SS', 3) 2000
  ('SS', 4) 2500
  ('SK', 1) 1000
  ('SK', 2) 1000
  ('SK', 3) 1500
  ('SK', 4) 2000
q
  US 0.25
  SS 0.2
  SK 0.1
p
  US 0.1
  SS 0.05
  SK 0.05
o
  US 500
  SS 800
  SK 500
n
  US 1500
  SS 2000
  SK 3000
m
  US 500
  SS 400
  SK 400
l
  US 200
  SS 500
  SK 500
k
  US 400
  SS 500
  SK 0


# Refinery Optimisation

In [4]:
mdl = AbstractModel()

## Sets

## Parameters

In [ ]:
DISTILLATION

cra --> light naphta
cra --> medium naphta
cra --> heavy naphta
cra --> light oil
cra --> heavy oil
cra --> residdum

crb --> light naphta
crb --> medium naphta
crb --> heavy naphta
crb --> light oil
crb --> heavy oil
crb --> residdum

REFORMING

light naphta --> reformed gasoline
medium naphta --> reformed gasoline
heavy naphta --> reformed gasoline

CRACKING

light oil --> jet fuel
heavy oil --> jet fuel

light oil --> fuel oil
heavy oil --> fuel oil

light oil --> cracked oil
heavy oil --> cracked oil

light oil --> cracked gasoline
heavy oil --> cracked gasoline

cracked oil --> fuel oil
cracked oil --> jet fuel
cracked gasoline --> petrol

residuum --> lube oil
residuum --> jet fuel
residuum --> fuel oil

BLENDING



## Variables

In [6]:
mdl.cra = Var(domain=NonNegativeReals)
mdl.crb = Var(domain=NonNegativeReals)
mdl.ln = Var(domain=NonNegativeReals)
mdl.mn = Var(domain=NonNegativeReals)
mdl.hn = Var(domain=NonNegativeReals)
mdl.lo = Var(domain=NonNegativeReals)
mdl.ho = Var(domain=NonNegativeReals)
mdl.r = Var(domain=NonNegativeReals)
mdl.lnrg = Var(domain=NonNegativeReals)
mdl.mnrg = Var(domain=NonNegativeReals)
mdl.hnrg = Var(domain=NonNegativeReals)
mdl.rg = Var(domain=NonNegativeReals)
mdl.locgo = Var(domain=NonNegativeReals)
mdl.hocgo = Var(domain=NonNegativeReals)
mdl.cg = Var(domain=NonNegativeReals)
mdl.co = Var(domain=NonNegativeReals)
mdl.lnpmf = Var(domain=NonNegativeReals)
mdl.lnrmf = Var(domain=NonNegativeReals)
mdl.mnpmf = Var(domain=NonNegativeReals)
mdl.mnrmf = Var(domain=NonNegativeReals)
mdl.hnpmf = Var(domain=NonNegativeReals)
mdl.hnrmf = Var(domain=NonNegativeReals)
mdl.rgpmf = Var(domain=NonNegativeReals)
mdl.rgrmf = Var(domain=NonNegativeReals)
mdl.cgpmf = Var(domain=NonNegativeReals)
mdl.cgrmf = Var(domain=NonNegativeReals)
mdl.lojf = Var(domain=NonNegativeReals)
mdl.hojf = Var(domain=NonNegativeReals)
mdl.rjf = Var(domain=NonNegativeReals)
mdl.cojf = Var(domain=NonNegativeReals)
mdl.rlbo = Var(domain=NonNegativeReals)
mdl.pmf = Var(domain=NonNegativeReals)
mdl.rmf = Var(domain=NonNegativeReals)
mdl.jf = Var(domain=NonNegativeReals)
mdl.fo = Var(domain=NonNegativeReals)
mdl.lbo = Var(domain=NonNegativeReals)

    'pyomo.core.base.var.SimpleVar'>) on block unknown with a new Component
    (type=<class 'pyomo.core.base.var.SimpleVar'>). This is usually indicative
    of a modelling error. To avoid this warning, use block.del_component() and
    block.add_component().


## Objective Function

In [ ]:
def max_profit(mdl):
    return sum(mdl.x[s,t] for s in mdl.S for t in mdl.T)

mdl.OBJ = Objective(rule=max_profit, sense=maximize)

## Constraints

**crude**

$cra\leq 20000$

$crb\leq 30000$

**distillation**

$cra+crb\leq 45000$

**reforming**

$lnrg+mnrg+hnrg\leq 10000$

**cracking**

$locgo+hocgo\leq 8000$

**lube-oil**

$500\leq lbo \leq 1000$

**distillation fractions**

$0.1*cra+0.15*crb=ln$

$0.2*cra+0.25*crb=mn$

$0.2*cra+0.18*crb=hn$

$0.12*cra+0.08*crb=lo$

$0.2*cra+0.19*crb=ho$

$0.13*cra+0.12*crb=r$

**reformed gasoline**

$0.6*lnrg+0.52*mnrg+0.45*hnrg=rg$

**cracked oil**

$0.68*locgo+0.75*hocgo=co$

**cracked gasoline**

$0.28*locgo+0.2*hocgo=cg$

**naphta**

$lnrg+lnpmf+lnrmf=ln$

$mnrg+mnpmf+mnrmf=mn$

$hnrg+hnpmf+hnrmf=hn$



In [1]:
def cra_expr(mdl):
    return mdl.cra <= 20000

mdl.cra_con = Constraint(rule=cra_expr)

def crb_expr(mdl):
    return mdl.crb <= 30000

mdl.crb_con = Constraint(rule=crb_expr)

def distillation_expr(mdl):
    return mdl.cra + mdl.crb <= 45000

mdl.distillation_con = Constraint(rule=distillation_expr)

def reforming_expr(mdl):
    return mdl.lnrg + mdl.mnrg + mdl.hnrg <= 10000

mdl.reforming_con = Constraint(rule=reforming_expr)

def cracking_expr(mdl):
    return mdl.locgo + mdl.hocgo <= 8000

mdl.cracking_con = Constraint(rule=cracking_expr)

def lube_oil_lb_expr(mdl):
    return mdl.lbo >= 500

mdl.lube_oil_lb_con = Constraint(rule=lube_oil_lb_expr)

def lube_oil_ub_expr(mdl):
    return mdl.lbo <= 1000

mdl.lube_oil_ub_con = Constraint(rule=lube_oil_ub_expr)

def light_naphta_mix_expr(mdl):
    return 0.1*mdl.cra + 0.15*mdl.crb == mdl.ln

mdl.light_naphta_mix_con = Constraint(rule=light_naphta_mix_expr)

def medium_naphta_mix_expr(mdl):
    return 0.2*mdl.cra + 0.25*mdl.crb == mdl.mn

mdl.medium_naphta_mix_con = Constraint(rule=medium_naphta_mix_expr)

def heavy_naphta_mix_expr(mdl):
    return 0.2*mdl.cra + 0.18*mdl.crb == mdl.hn

mdl.heavy_naphta_mix_con = Constraint(rule=heavy_naphta_mix_expr)

def light_oil_mix_expr(mdl):
    return 0.12*mdl.cra + 0.08*mdl.crb == mdl.lo

mdl.light_oil_mix_con = Constraint(rule=light_oil_mix_expr)

def heavy_oil_mix_expr(mdl):
    return 0.2*mdl.cra + 0.19*mdl.crb == mdl.ho

mdl.heavy_oil_mix_con = Constraint(rule=heavy_oil_mix_expr)

def residuum_mix_expr(mdl):
    return 0.13*mdl.cra + 0.12*mdl.crb == mdl.r

mdl.residuum_mix_expr = Constraint(rule=residuum_mix_expr)

def naphta_reformed_gasoline_expr(mdl):
    return 0.6*mdl.lnrg + 0.52*mdl.mnrg + 0.45*mdl.hnrg == mdl.rg

mdl.naphta_reformed_gasoline_con = Constraint(rule=naphta_reformed_gasoline_expr)

def cracked_oil_expr(mdl):
    return 0.68*mdl.locgo + 0.75*mdl.hocgo == mdl.co

mdl.cracked_oil_con = Constraint(rule=cracked_oil_expr)

def cracked_gasoline_expr(mdl):
    return 0.28*mdl.locgo + 0.2*mdl.hocgo == mdl.cg

mdl.cracked_gasoline_con = Constraint(rule=cracked_gasoline_expr)

def light_naphta_expr(mdl):
    return mdl.lnrg + mdl.lnpmf + mdl.lnrmf == mdl.ln

mdl.light_naphta_con = Constraint(rule=light_naphta_expr)

def medium_naphta_expr(mdl):
    return mdl.mnrg + mdl.mnpmf + mdl.mnrmf == mdl.mn

mdl.medium_naphta_con = Constraint(rule=medium_naphta_expr)

def heavy_naphta_expr(mdl):
    return mdl.hnrg + mdl.hnpmf + mdl.hnrmf == mdl.hn

mdl.heavy_naphta_con = Constraint(rule=heavy_naphta_expr)

NameError: name 'Constraint' is not defined